# Data Cleaning

---

# Overview

In this file we clean and merge four datasets:
- Transfermarkt webscrapped data
- Tables containing week-by-week prices of each player
- Tables containing all player statistics (goals, assists, yellow cards, etc.)
- Elo ratining

In [22]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from unidecode import unidecode

# Import Prices tables

In [23]:
def import_prices (season):

    '''
    Imports prices time series for each player in a given season
    Output is a single dataframe
    Expects as argument the first year of the season in format yy (for example, 19 to import season 2019-20)
    '''

    df = pd.read_excel("prices/Prices Season {}-{}.xlsx".format(season, season+1))
    df.insert(loc=4, column="season", value=2000+season)
    df = df.drop(["quotazioneMantra", "fantaVoto"], axis=1) # the former does not concern our project, the latter is the result implied by the whole scores dfs
    df.columns = ["id", "name", "role", "team", "season", "matchday", "price"]
    df = df.set_index("id")

    return df

In [24]:
prices16 = import_prices(16)
prices17 = import_prices(17)
prices18 = import_prices(18)
prices19 = import_prices(19)

prices = pd.concat([prices16, prices17, prices18, prices19])
prices.reset_index(inplace=True)

In [25]:
prices_list = []
for index,row in prices.iterrows():
    temp = list(zip(row['matchday'].split(','), row['price'].split(',')))
    for touple in temp:
        prices_list.append([row['name'], row['season'], touple[0], touple[1]])

In [26]:
price = pd.DataFrame(prices_list,  columns=['player', 'season', 'matchweek', 'price'])
price['season']    = price['season'].apply(lambda x: int(x))
price['matchweek'] = price['matchweek'].apply(lambda x: int(x))
price['price']     = price['price'].apply(lambda x: float(x))
price['player']    = price['player'].apply(lambda x: x.upper())

In [27]:
# Sometimes player name has an apostrophie at the end, removing it improves the merge
# with transfermarkt table
def remove_apostrophe(text):
    if text[-1] == "'":
        return text[:-1]
    else:
        return text

price['player'] = price['player'].apply(lambda x: remove_apostrophe(x))

OBSERVING PRICE DF

*   "id" is unique to each player, can be used in place of "name" to join prices and scores dfs
*   we have prices only for matchdays in which the player touched the pitch; for the rest the prices remain the same (to be confirmed) but it might be useful to store arrays all of the same lenght
*   "matchday" and "prices" already imply the number of games played by each player at each point of the season: counting within these arrays should be more efficient than counting in the whole scores df





# Import Scores

In [28]:
def import_scores_matchday (season, matchday):

    # import the data, change the headers and add three empty columns: "team", "coach" and "coach_score"
    # in season 2019-20 they started dividing in two columns assists during action and on free-kick/corner: to account for that, we add a columns full of 0s in the years before
    df = pd.read_excel(f"Scores/Scores 20{season}-{season+1}/Voti_Fantacalcio_Stagione_20{season}-{season+1}_Giornata_{matchday}.xlsx")
    df = df.drop([0,1,2]) # drop the headers of the document
    if (season != 19) and (season != 20):
        df["assist_fk"] = 0
        df.columns = ["id","role","player","base_score","goal_scored","goal_conceded","penalty_saved","penalty_failed", "penalty_scored", "own_goal","yellow_card","red_card","assist","goal_decisive_draw","goal_decisive_win","assist_fk"]
    else:
        df.columns = ["id","role","player","base_score","goal_scored","goal_conceded","penalty_saved","penalty_failed", "penalty_scored", "own_goal","yellow_card","red_card","assist","assist_fk","goal_decisive_draw","goal_decisive_win"]
    df["team"] = np.nan
    df["coach"] = np.nan
    df["coach_score"] = np.nan

    # we want team's name and coach's name and score to appear in the row of each player
    # currently: team appears as header above its players and coach; the coach always represents the last row of each team; a second header appears under the name of each team, above its players and coach
    # thus, we can retrieve indeces of headers and coaches and use those to delimit the rows which refer to each team
    # when the info have been copied to the players' rows, we can delete the headers and the coaches' rows
    team_delimiters = zip(df[df["id"]=="Cod."].index, df[df["role"]=="ALL"].index)
    for team, coach in team_delimiters:
        team_name = df["id"].loc[team-1]
        coach_name = df["player"].loc[coach]
        coach_score = df["base_score"].loc[coach]
        df.loc[team+1:coach, "team"] = team_name
        df.loc[team+1:coach, "coach"] = coach_name
        df.loc[team+1:coach, "coach_score"] = coach_score
        df = df.drop([team-1, team, coach])
    
    # now that the dataframe is formatted correctly, we sum the two columns for assists (in most cases, one is made of 0s, we added it some lines above) and keep only one
    df["assist"] = df["assist"] + df["assist_fk"]
    df = df.drop(["assist_fk"], axis=1)
        
    # modify the format of player's name and team in preparation to join with TransferMarkt data
    df["team"] = df["team"].apply(lambda x: x.upper())
    #df["player"] = df["player"].apply(lambda x: x.capitalize() if x == "ZUCULINI F" or "ZUCULINI B" else (x.capitalize() if len(x.split(" ")) == 1 else x.capitalize() if len(x.split(" ")[1]) <= 2 else x.split(" ")[1].capitalize())))
    #df["player"] = df["player"].apply(lambda x: x.capitalize() if len(x.split(" ")) == 1 else (x.capitalize() if len(x.split(" ")[1]) <= 2 else x.split(" ")[1].capitalize()))
    #df = avoid_duplicate_names(df)
    
    # add columns reporting season and matchday
    df["season"] = 2000+season
    df["matchweek"] = matchday

    # translate column "role", as the abbreviations refer to italian words
    df.loc[df["role"] == "P", "role"] = "G"
    df.loc[df["role"] == "C", "role"] = "M"

    return df

In [29]:
def import_scores_season (season):

  '''
  Uses function "import_scores_matchday" 38 times to import all matchdays in one season
  Output is a single dataframe
  Expects as argument the first year of the season in format yy (for example, 19 to import season 2019-20)
  '''

  df = pd.DataFrame()
  
  # import and clean the scores of each matchday; add columns "season" and "matchday"
  # then, concatenate all matchdays of one season in a single df
  for matchday in range(38): #38 matchdays for each season
    df_matchday = import_scores_matchday(season, matchday+1)
    df = pd.concat([df, df_matchday])
  
  return df.sort_index()

In [30]:
scores16 = import_scores_season(16)
scores17 = import_scores_season(17)
scores18 = import_scores_season(18)
scores19 = import_scores_season(19)

scores = pd.concat([scores16, scores17, scores18, scores19])
scores.head(2)

,id,role,player,base_score,goal_scored,goal_conceded,penalty_saved,penalty_failed,penalty_scored,own_goal,yellow_card,red_card,assist,goal_decisive_draw,goal_decisive_win,team,coach,coach_score,season,matchweek
5,4,G,SPORTIELLO,4.5,0,4,0,0,0,0,0,0,0,0,0,ATALANTA,GASPERINI,5.5,2016,1
5,4,G,SPORTIELLO,6,0,1,0,0,0,0,0,0,0,0,0,ATALANTA,GASPERINI,7.0,2016,18


In [31]:
#1 null value for 'player' <- remove that row
scores = scores[scores.player.notnull()]

# Sometimes player name has an apostrophie at the end, removing it improves the merge
# with transfermarkt table
scores['player'] = scores['player'].apply(lambda x: remove_apostrophe(x))

OBSERVING SCORES DF

*   "base_score" is not entirely made of floats: some 6* appear. That happens when a player enters the pitch for just a few minutes and is therefore not possible to evaluate his performance.
Normally, their scores are not counted, but if they manage to get a yellow card in those few minutes, they are attributed 5.5.
The column needs to be cleaned and we need to decide whether to count 6* within players appearences or not.
*   "goal_conceded" and "penalty_saved" of course concern only goalkeepers ("role" == G).
*   in "coach" we might retain coach's unique id, instead of name, but we'd need to be careful not to treat that as a continous variable.
The same goes for "team", except in that case we'd also need to generate an index ourselves.
*   "role" might change from one year to the other: it's not frequent, but it still happens. We need to track a couple players that switched role within the training dataset and make sure the model does not screw up on that.
*   "goal_decisive_draw"/"goal_decisive_win" are binary variables representing whether the goal scored by the player was the last one scored in a drawn match/was the n+1 goal of the winning team (where n is the number of goals scored by the losing team). Some people associate a small bonus to them, which I would avoid for the scope of the project, as that would increase volatility of players' scores. Nevertheless, I left them there as they might turn out to be useful as predictors.






# Import Data from TransferMarkt

In [32]:
transfermarkt = pd.read_csv("ready_datasets/transfermarkt_data.csv")

In [33]:
def modify_player (data):
    
    # the various conditions of this first lambda function take care of a few players who both have the same name and played in the same teams
    # the last else statement is the more general condition that holds for most names: the player's names are split on the last whitespace to select the surname and stripped of accents using the function unidecode
    data["player"] = transfermarkt["player"].apply(lambda x: "ZUCULINI F" if x == "Franco Zuculini" else ("ZUCULINI B" if x == "Bruno Zuculini" else ("PELLEGRINI LO" if x == "Lorenzo Pellegrini" else ("PELLEGRINI LU" if x == "Luca Pellegrini" else ("FELIPE ANDERSON" if x == "Felipe Anderson" else ("ANDERSON A" if x == "André Anderson" else ("ANDERSON D" if x == "Djavan Anderson" else (unidecode(unidecode(x.split(" ")[-1], "utf-8").replace("'","").upper())))))))))
    
    # there a few cases in which the general rule does not work, because the last name is formed by two words, often by a prefix
    # the easiest solution is to address these manually
    data.loc[(data["player"] == "ALONSO"), "player"] = "MARCOS ALONSO"
    data.loc[(data["player"] == "VRIJ"), "player"] = "DE VRIJ"
    data.loc[(data["player"] == "AVEST"), "player"] = "TER AVEST"
    data.loc[(data["player"] == "BECAO"), "player"] = "RODRIGO BECAO"
    data.loc[(data["player"] == "CARMINE"), "player"] = "DI CARMINE"
    data.loc[(data["player"] == "CAVIGLIA"), "player"] = "NICOLUSSI CAVIGLIA"
    data.loc[(data["player"] == "PAUL"), "player"] = "DE PAUL"    
    data.loc[(data["player"] == "MARIO"), "player"] = "JOAO MARIO"
    data.loc[(data["player"] == "ADRIANO"), "player"] = "LUIZ ADRIANO"
    data.loc[(data["player"] == "ALBERTO"), "player"] = "LUIS ALBERTO"    
    data.loc[(data["player"] == "VALERO"), "player"] = "BORJA VALERO"
    data.loc[(data["player"] == "FALQUE"), "player"] = "IAGO FALQUE"
    data.loc[(data["player"] == "ROON"), "player"] = "DE ROON"
    data.loc[(data["player"] == "VELOSO"), "player"] = "MIGUEL VELOSO"    
    data.loc[(data["player"] == "LARSEN"), "player"] = "STRYGER LARSEN"    
    data.loc[(data["player"] == "SHAARAWY"), "player"] = "EL SHAARAWY" 
    data.loc[(data["player"] == "LEIVA"), "player"] = "LUCAS LEIVA"    
    data.loc[(data["player"] == "JESUS"), "player"] = "JUAN JESUS"    
    data.loc[(data["player"] == "MAIO"), "player"] = "DE MAIO"
    data.loc[(data["player"] == "RUI"), "player"] = "MARIO RUI"
    data.loc[(data["player"] == "LORENZO"), "player"] = "DI LORENZO"    
    data.loc[(data["player"] == "SCIGLIO"), "player"] = "DE SCIGLIO"    
    data.loc[(data["player"] == "PERES"), "player"] = "BRUNO PERES"    
    data.loc[(data["player"] == "HUGO"), "player"] = "VITOR HUGO"    
    data.loc[(data["player"] == "HENRIQUE"), "player"] = "BRUNO HENRIQUE"    
    data.loc[(data["player"] == "LIGT"), "player"] = "DE LIGT"
    data.loc[(data["player"] == "GUZMAN"), "player"] = "DE GUZMAN"
    data.loc[(data["player"] == "DIAS"), "player"] = "GIL DIAS"    
    data.loc[(data["player"] == "GUMINA"), "player"] = "LA GUMINA"     
    data.loc[(data["player"] == "KADDOURI"), "player"] = "EL KADDOURI"    
    data.loc[(data["player"] == "SOSA"), "player"] = "JOSE SOSA"    
    data.loc[(data["player"] == "GAUDIO"), "player"] = "DI GAUDIO"
    data.loc[(data["player"] == "FASO"), "player"] = "LO FASO"
    data.loc[(data["player"] == "FLORES"), "player"] = "FLORO FLORES"
    data.loc[(data["player"] == "MANTIA"), "player"] = "LA MANTIA"
    data.loc[(data["player"] == "YAMIQ"), "player"] = "EL YAMIQ"
    data.loc[(data["player"] == "EVANGELISTA"), "player"] = "LUCAS EVANGELISTA"
    data.loc[(data["player"] == "MELO"), "player"] = "FELIPE MELO"
    data.loc[(data["player"] == "MORENO"), "player"] = "HECTOR MORENO"
    data.loc[(data["player"] == "WIEL"), "player"] = "VAN DER WIEL"
    data.loc[(data["player"] == "SOARES"), "player"] = "CEDRIC SOARES"
    data.loc[(data["player"] == "JORDAO"), "player"] = "BRUNO JORDAO"
    data.loc[(data["player"] == "TAVARES"), "player"] = "JUNIOR TAVARES"
    data.loc[(data["player"] == "CABRAL"), "player"] = "RAFAEL CABRAL"
    data.loc[(data["player"] == "CRUZ"), "player"] = "DA CRUZ"
    data.loc[(data["player"] == "PEREZ"), "player"] = "CARLES PEREZ"    
    data.loc[(data["player"] == "PINTO"), "player"] = "DEL PINTO"
    data.loc[(data["player"] == "FELIPPE", "player")] = "LUCAS FELIPPE"
    
    # the list that follows takes care of those last names that are duplicated, but for which the combination name + team is still a unique identifier
    # once again, the easiest solution is to address these cases manually
    data.loc[(data["player"] == "FERNANDES") & (data["team"] == "SAMPDORIA"), "player"] = "BRUNO FERNANDES"    
    data.loc[(data["player"] == "SILVESTRI") & (data["team"] == "TORINO"), "player"] = "DE SILVESTRI"
    data.loc[(data["player"] == "OLSEN") & (data["team"] == "BOLOGNA"), "player"] = "SKOV OLSEN"   
    data.loc[(data["player"] == "RODIRGUEZ") & (data["team"] == "CHIEVO"), "player"] = "RODRIGUEZ A"
    data.loc[((data["player"] == "COSTA") & (data["team"] == "CHIEVO")) | ((data["player"] == "COSTA") & (data["team"] == "SPAL")), "player"] = "COSTA F"
    data.loc[(data["player"] == "RADU") & (data["team"] == "GENOA"), "player"] = "RADU I"
    data.loc[(data["player"] == "PEREIRA") & (data["team"] == "JUVENTUS"), "player"] = "MATHEUS PEREIRA"
    data.loc[(data["player"] == "ALVES") & (data["team"] == "JUVENTUS"), "player"] = "DANI ALVES"
    data.loc[(data["player"] == "GOMEZ") & (data["team"] == "MILAN"), "player"] = "GOMEZ G"
    data.loc[(data["player"] == "ROSSI") & (data["team"] == "LAZIO"), "player"] = "ROSSI A"
    data.loc[(data["player"] == "SILVA") & (data["team"] == "PALERMO"), "player"] = "STEFAN SILVA"
    data.loc[((data["player"] == "SIMIC") & (data["team"] == "SAMPDORIA")) | ((data["player"] == "SIMIC") & (data["team"] == "SPAL")), "player"] = "SIMIC L"
    data.loc[(data["player"] == "RODRIGUEZ") & (data["team"] == "MILAN"), "player"] = "RODRIGUEZ R"
    data.loc[((data["player"] == "MILINKOVIC-SAVIC") & (data["team"] == "Torino")) | ((data["player"] == "MILINKOVIC-SAVIC") & (data["team"] == "SPAL")), "player"] = "MILINKOVIC-SAVIC V"
    data.loc[(data["player"] == "CODA") & (data["team"] == "BENEVENTO"), "player"] = "CODA M"
    data.loc[(data["player"] == "COSTA") & (data["team"] == "JUVENTUS"), "player"] = "DOUGLAS COSTA"
    data.loc[(data["player"] == "ROSSI") & (data["team"] == "ATALANTA"), "player"] = "ROSSI F"
    data.loc[(data["player"] == "KEITA") & (data["team"] == "BOLOGNA"), "player"] = "KEITA C"
    data.loc[(data["player"] == "NETO") & (data["team"] == "LAZIO"), "player"] = "PEDRO NETO"
    data.loc[(data["player"] == "FELIPE") & (data["team"] == "LAZIO"), "player"] = "LUIZ FELIPE"
    data.loc[(data["player"] == "LOPEZ") & (data["team"] == "INTER"), "player"] = "LISANDRO LOPEZ"
    data.loc[(data["player"] == "SILVA") & (data["team"] == "ROMA"), "player"] = "SILVA J"
    data.loc[(data["player"] == "MARTINEZ") & (data["team"] == "INTER"), "player"] = "MARTINEZ L"
    data.loc[(data["player"] == "BERISHA") & (data["team"] == "LAZIO"), "player"] = "BERISHA V"
    data.loc[(data["player"] == "RIGONI") & (data["team"] == "ATALANTA"), "player"] = "RIGONI E"
    data.loc[(data["player"] == "VERDE") & (data["team"] == "CAGLIARI"), "player"] = "VERDE F"
    data.loc[(data["player"] == "TELLO") & (data["team"] == "FIORENTINA"), "player"] = "TELLO C"
    data.loc[(data["player"] == "ROMERO") & (data["team"] == "CROTONE"), "player"] = "ROMERO A"
    data.loc[(data["player"] == "SANDRO") & (data["team"] == "JUVENTUS"), "player"] = "ALEX SANDRO"
    data.loc[(data["player"] == "COSTA") & (data["team"] == "BOLOGNA"), "player"] = "DA COSTA"
    data.loc[(data["player"] == "ROMAGNOLI") & (data["team"] == "BOLOGNA"), "player"] = "ROMAGNOLI S"
    data.loc[(data["player"] == "RODRIGUEZ") & (data["team"] == "FIORENTINA"), "player"] = "RODRIGUEZ GO"
    data.loc[(data["player"] == "ALVAREZ") & (data["team"] == "SAMPDORIA"), "player"] = "ALVAREZ R"
    data.loc[((data["player"] == "ALVES") & (data["team"] == "PARMA")) | ((data["player"] == "ALVES") & (data["team"] == "CAGLIARI")), "player"] = "BRUNO ALVES"
    data.loc[(data["player"] == "ANDERSON") & (data["team"] == "LAZIO"), "player"] = "FELIPE ANDERSON"
    data.loc[((data["player"] == "BALDE") & (data["team"] == "SAMPDORIA")) | ((data["player"] == "BALDE") & (data["team"] == "LAZIO")) | ((data["player"] == "BALDE") & (data["team"] == "INTER")), "player"] = "KEITA B"
    data.loc[(data["player"] == "BALOGH") & (data["team"] == "PALERMO"), "player"] = "BALOGH N"
    data.loc[(data["player"] == "BERISHA") & (data["team"] == "SPAL"), "player"] = "BERISHA E"
    data.loc[(data["player"] == "CARACCIOLO") & (data["team"] == "VERONA"), "player"] = "CARACCIOLO A"
    data.loc[(data["player"] == "CHIARA") & (data["team"] == "BENEVENTO"), "player"] = "DI CHIARA"
    data.loc[(data["player"] == "CIOFANI") & (data["team"] == "FROSINONE"), "player"] = "CIOFANI D"
    data.loc[(data["player"] == "CISSE") & (data["team"] == "BENEVENTO"), "player"] = "CISSE K"
    data.loc[(data["player"] == "COLOMBO") & (data["team"] == "CAGLIARI"), "player"] = "COLOMBO R"
    data.loc[(data["player"] == "DONNARUMMA") & (data["team"] == "MILAN"), "player"] = "DONNARUMMA G"
    data.loc[(data["player"] == "PEZZELLA") & (data["team"] == "FIORENTINA"), "player"] = "PEZZELLA GER"
    data.loc[((data["player"] == "ZAPATA") & (data["team"] == "UDINESE")) | ((data["player"] == "ZAPATA") & (data["team"] == "SAMPDORIA")) | ((data["player"] == "ZAPATA") & (data["team"] == "ATALANTA")), "player"] = "ZAPATA D"
    data.loc[((data["player"] == "FRANCESCO") & (data["team"] == "BOLOGNA")) | ((data["player"] == "FRANCESCO") & (data["team"] == "SASSUOLO")) | ((data["player"] == "FRANCESCO") & (data["team"] == "SPAL")), "player"] = "DI FRANCESCO F"
    data.loc[((data["player"] == "FERRARI") & (data["team"] == "CROTONE")) | ((data["player"] == "FERRARI") & (data["team"] == "SAMPDORIA")) | ((data["player"] == "FERRARI") & (data["team"] == "SASSUOLO")), "player"] = "FERRARI G"
    data.loc[(data["player"] == "LUKAKU") & (data["team"] == "LAZIO"), "player"] = "LUKAKU J"
    data.loc[((data["player"] == "TRAORE") & (data["team"] == "EMPOLI")) | ((data["player"] == "TRAORE") & (data["team"] == "SASSUOLO")), "player"] = "TRAORE HJ"
    data.loc[((data["player"] == "ZAPATA") & (data["team"] == "MILAN")) | ((data["player"] == "ZAPATA") & (data["team"] == "GENOA")), "player"] = "ZAPATA C"
    data.loc[((data["player"] == "PEZZELLA") & (data["team"] == "PALERMO")) | ((data["player"] == "PEZZELLA") & (data["team"] == "UDINESE")) | ((data["player"] == "PEZZELLA") & (data["team"] == "GENOA")) | ((data["player"] == "PEZZELLA") & (data["team"] == "PARMA")), "player"] = "PEZZELLA GIU"
    data.loc[((data["player"] == "GONZALEZ") & (data["team"] == "PALERMO")) | ((data["player"] == "GONZALEZ") & (data["team"] == "BOLOGNA")), "player"] = "GONZALEZ G"
    data.loc[(data["player"] == "RIGONI") & (data["team"] == "CHIEVO"), "player"] = "RIGONI N"
    data.loc[(data["player"] == "GOMIS") & (data["team"] == "SPAL"), "player"] = "GOMIS A"
    data.loc[((data["player"] == "FERRARI") & (data["team"] == "BOLOGNA")) | ((data["player"] == "FERRARI") & (data["team"] == "VERONA")) | ((data["player"] == "FERRARI") & (data["team"] == "SAMPDORIA")), "player"] = "FERRARI A"
    data.loc[((data["player"] == "RICCI") & (data["team"] == "SASSUOLO")) | ((data["player"] == "RICCI") & (data["team"] == "CROTONE")) | ((data["player"] == "RICCI") & (data["team"] == "GENOA")), "player"] = "RICCI F"
    data.loc[(data["player"] == "SANCHEZ") & (data["team"] == "FIORENTINA"), "player"] = "SANCHEZ C"
    data.loc[(data["player"] == "CAN") & (data["team"] == "JUVENTUS"), "player"] = "CAN " #cialtroni...
    data.loc[((data["player"] == "LOPEZ") & (data["team"] == "UDINESE")) | ((data["player"] == "LOPEZ") & (data["team"] == "TORINO")), "player"] = "MAXI LOPEZ"
    data.loc[(data["player"] == "HERNANDEZ") & (data["team"] == "MILAN"), "player"] = "HERNANDEZ T"
    data.loc[((data["player"] == "PEREIRA") & (data["team"] == "SAMPDORIA")) | ((data["player"] == "PEREIRA") & (data["team"] == "GENOA")), "player"] = "PEREIRA P"
    data.loc[(data["player"] == "LOPEZ") & (data["team"] == "ROMA"), "player"] = "PAU LOPEZ"
    data.loc[(data["player"] == "LEAO") & (data["team"] == "MILAN"), "player"] = "RAFAEL LEAO"
    data.loc[(data["player"] == "DONNARUMMA") & (data["team"] == "BRESCIA"), "player"] = "DONNARUMMA AL"
    data.loc[(data["player"] == "OLIVERA") & (data["team"] == "FIORENTINA"), "player"] = "OLIVERA M"
    data.loc[(data["player"] == "EMERSON") & (data["team"] == "ROMA"), "player"] = "EMERSON P"
    data.loc[((data["player"] == "RIGONI") & (data["team"] == "GENOA")) | ((data["player"] == "RIGONI") & (data["team"] == "PARMA")), "player"] = "RIGONI L"
    data.loc[((data["player"] == "GENNARO") & (data["team"] == "CAGLIARI")) | ((data["player"] == "GENNARO") & (data["team"] == "LAZIO")), "player"] = "DI GENNARO D"
    data.loc[(data["player"] == "LUIZ") & (data["team"] == "SPAL"), "player"] = "EVERTON LUIZ"
    data.loc[((data["player"] == "MAURI") & (data["team"] == "MILAN")) | ((data["player"] == "MAURI") & (data["team"] == "EMPOLI")), "player"] = "MAURI J"
    data.loc[((data["player"] == "PAVLOVIC") & (data["team"] == "CROTONE")) | ((data["player"] == "PAVLOVIC") & (data["team"] == "")), "player"] = "PAVLOVIC D"
    data.loc[((data["player"] == "FERRARI") & (data["team"] == "BOLOGNA")) | ((data["player"] == "FERRARI") & (data["team"] == "VERONA")) | ((data["player"] == "FERRARI") & (data["team"] == "SAMPDORIA")), "player"] = "FERRARI A"
    data.loc[(data["player"] == "SILVA") & (data["team"] == "MILAN"), "player"] = "ANDRE SILVA"
    data.loc[((data["player"] == "SALCEDO") & (data["team"] == "GENOA")) | ((data["player"] == "SALCEDO") & (data["team"] == "VERONA")) | ((data["player"] == "SALCEDO") & (data["team"] == "UDINESE")), "player"] = "SALCEDO E"
    data.loc[(data["player"] == "GASPAR") & (data["team"] == "FIORENTINA"), "player"] = "GASPAR B"
    data.loc[(data["player"] == "FERNANDEZ") & (data["team"] == "MILAN"), "player"] = "FERNANDEZ M"
    data.loc[(data["player"] == "DANILO") & (data["team"] == "UDINESE"), "player"] = "DANILO LAR"  
    data.loc[(data["player"] == "ROSSI") & (data["team"] == "ROMA"), "player"] = "DE ROSSI"
    data.loc[(data["player"] == "PEDRO") & (data["team"] == "CAGLIARI"), "player"] = "JOAO PEDRO"    
    data.loc[(data["player"] == "ORBAN") & (data["team"] == "GENOA"), "player"] = "ORBAN L"
    data.loc[(data["player"] == "COULIBALY") & (data["team"] == "PESCARA"), "player"] = "COULIBALY M"
    data.loc[(data["player"] == "SIMIC") & (data["team"] == "CROTONE"), "player"] = "SIMIC S"
    data.loc[(data["player"] == "DIONISI") & (data["team"] == "FROSINONE"), "player"] = "DIONISI F"
    data.loc[(data["player"] == "VIVIANI") & (data["team"] == "BRESCIA"), "player"] = "VIVIANI M"
    data.loc[((data["player"] == "RIGONI") & (data["team"] == "ATALANTA")) | ((data["player"] == "RIGONI") & (data["team"] == "SAMPDORIA")), "player"] = "RIGONI E"
    data.loc[(data["player"] == "DODO") & (data["team"] == "SAMPDORIA"), "player"] = "DODO R"
    data.loc[(data["player"] == "MURIC") & (data["team"] == "PESCARA"), "player"] = "MURIC R"
    data.loc[(data["player"] == "SILVA") & (data["team"] == "UDINESE"), "player"] = "GABRIEL SILVA"
    data.loc[(data["player"] == "COLLEY") & (data["team"] == "ATALANTA"), "player"] = "COLLEY E"
    data.loc[(data["player"] == "HENDERSON") & (data["team"] == "VERONA"), "player"] = "HENDERSON L"
    data.loc[(data["player"] == "PEDRO") & (data["team"] == "FIORENTINA"), "player"] = "PEDRO G"
    data.loc[(data["player"] == "RANIERI") & (data["team"] == "FIORENTINA"), "player"] = "RANIERI L"
    data.loc[(data["player"] == "VITALE") & (data["team"] == "VERONA"), "player"] = "VITALE L"
    data.loc[(data["player"] == "THIAM") & (data["team"] == "SPAL"), "player"] = "THIAM D"
              
    return data

In [34]:
#modify the format of the player's name and team to match that of Fantacalcio data

transfermarkt = transfermarkt[transfermarkt["season"].isin([2016, 2017, 2018, 2019])]

#there is no easy way to write a function that modifies the name of the teams: being few values we'll simply use a dictionary for that
team_names = {"Juventus FC":"JUVENTUS", "AS Roma":"ROMA", "Inter Milan":"INTER", "SSC Napoli":"NAPOLI", "AC Milan":"MILAN",
              "SS Lazio":"LAZIO", "ACF Fiorentina":"FIORENTINA", "UC Sampdoria":"SAMPDORIA", "Genoa CFC":"GENOA",
              "Udinese Calcio":"UDINESE", "Torino FC":"TORINO", "Bologna FC 1909":"BOLOGNA", "US Sassuolo":"SASSUOLO",
              "Atalanta BC":"ATALANTA", "Athletic Carpi 2021":"CARPI", "Palermo FC":"PALERMO", "US Palermo":"PALERMO",
              "Hellas Verona":"VERONA", "Chievo Verona":"CHIEVO", "Frosinone Calcio":"FROSINONE", "FC Empoli":"EMPOLI",
              "Cagliari Calcio":"CAGLIARI", "Delfino Pescara 1936":"PESCARA", "FC Crotone":"CROTONE",
              "Benevento Calcio":"BENEVENTO", "Parma Calcio 1913":"PARMA", "Brescia Calcio":"BRESCIA",
              "US Lecce":"LECCE", "Spezia Calcio":"SPEZIA"}

transfermarkt = transfermarkt.replace(team_names)

transfermarkt = modify_player(transfermarkt)

transfermarkt.head(2)

,player,team,matchweek,match_date,home_team,away_team,score,status,season
26450,NETO,JUVENTUS,1,"Aug 20, 2016",Juventus,Fiorentina,2:1,on the bench,2016
26451,NETO,JUVENTUS,2,"Aug 27, 2016",Lazio,Juventus,0:1,on the bench,2016


Create player_status column

In [35]:
#Find possible positions of players (these values mean a player has played in the game)
possibe_positions = transfermarkt[transfermarkt['status'].apply(lambda x: len(str(x))) <= 2]['status'].value_counts().index.tolist()
print(possibe_positions)

['CB', 'CM', 'CF', 'DM', 'GK', 'LB', 'RB', 'AM', 'LW', 'LM', 'RW', 'RM', 'SS', 'SW']


In [36]:
def player_status(row):
        if row['status'] == 'Red card suspension' or row['status'] == 'Indirect card suspension' or row['status'] == 'Yellow card suspension' or row['status'] == 'Suspended' or row['status'] == 'Suspension through sports court':
            return 'suspended'
        elif row['status']== 'on the bench':
            return 'on the bench'
        elif row['status'] == 'Not in squad':
            return 'not in squad'
        elif row['status'] in possibe_positions:
            return 'played in the game'
        elif str(row['status']) == 'nan':
            return 'error_status'
        else:
            return 'injured'

In [37]:
#Run this to create a new column for injuries table status categorization
transfermarkt['player_status']= np.NaN
transfermarkt['player_status']= transfermarkt.apply(lambda row: player_status(row), axis=1)

In [38]:
transfermarkt['player_status'].value_counts()

played in the game    41354
on the bench          27094
not in squad          21807
injured                9892
error_status           5379
suspended              1817
Name: player_status, dtype: int64

# Merge Prices, Scores and Transfermark data

In [39]:
print('Rows in trnasfermarkt data: ', len(transfermarkt))
print('Rows in scores data:        ', len(scores))
# print('Rows in prices data: 'len())

Rows in trnasfermarkt data:  107343
Rows in scores data:         42683


In [40]:
# merge data from Fantacalcio.it and transfermarkt.com
data = pd.merge(transfermarkt, scores,  how='left', left_on=["player", "team", "season", "matchweek"], right_on=["player", "team", "season", "matchweek"])

In [41]:
print('New DF:', len(data))
print('This many scores transferred: ', len(data[data['base_score'].apply(lambda x: str(x)) != 'nan']['base_score']))
print('This many we should have filled: ', len(transfermarkt[transfermarkt['player_status'] == 'played in the game']))
data2 = pd.merge(scores, data, how='left', left_on=["player", "team", "season", "matchweek"], right_on=["player", "team", "season", "matchweek"])
len_unused = len(data2[data2['base_score_y'].isnull()])
print('Unused from Scores:', len_unused)

New DF: 107343
This many scores transferred:  40369
This many we should have filled:  41354
Unused from Scores: 2357


In [42]:
# impute the NaNs in the columns "id" and "role" in the table resulting from the merge

#create tables to easily retrieve the info of each player during each season
players_info = scores.groupby(["id", "season", "role"]).agg('first').reset_index()
players_info = players_info[["id", "player", "role", "season"]]

#use the table above to impute the NaNs in the table resulting from the join
#.TOLIST()[0] IS LESS EFFICIENT THAN .ITEM() BUT AVOIDS RAISING A VALUEERROR FOR MISSING CORRESPONDANCES BETWEEN THE NAMES IN THE TWO DATASETS: IF WE MANAGE TO CORRECT EVERYTHING CAN BE CHANGED
data["id"]   = data.apply(lambda x: players_info.loc[(players_info.player == x.player) & (players_info.season == x.season), "id"].tolist()[0]   if len(players_info.loc[(players_info.player == x.player) & (players_info.season == x.season), "id"].tolist()) > 0 else np.NaN, axis=1)
data["role"] = data.apply(lambda x: players_info.loc[(players_info.player == x.player) & (players_info.season == x.season), "role"].tolist()[0] if len(players_info.loc[(players_info.player == x.player) & (players_info.season == x.season), "role"].tolist()) > 0 else np.NaN, axis=1)

**Duplicate rows issue**

In [43]:
# Some rows do not have unique [player, season, matchweek] which causes problems later on
# Mostly this is because one player was assigned to two teams or counted twice per game
# The below only takes first row of a grouped sorted database (manually check it picks the right row ~90% of time)

# Deal with particular cases of duplicate [player, team, season, matchweek]
data = data.sort_values(by=["player", "team", "season", "matchweek", "base_score"]).groupby(by=["player", "team", "season", "matchweek"]).first().reset_index()

# Helper column for dealing with duplicate [player, season, matchweek]
# Shows 0 if col 'team' is neither in 'home team' nor in 'away team' <- if given the chance we will get rid of this row
data['team_check'] = [ -1 if ((row['team'].lower() == row['home_team'].lower()) | (row['team'].lower() == row['away_team'].lower())) else 0 for index,row in data.iterrows()]

# Deal with cases of duplicate [player, season, matchweek]
data = data.sort_values(by=["player", "season", "matchweek", "base_score", 'team_check']).groupby(by=["player", "season", "matchweek"]).first().reset_index()

#check
len(data[data.duplicated(["player", "season", "matchweek"])])

0

The above solution has some flaws e.g.: Player has duplicate [player, season, match] for 2 teams. In a game had no score for either team (e.g. injured), we picked the team that appeared in 'away team' or 'home team'. In case when the wrong team was playing against the correct team, both appeared in 'away team' or 'home team'. In this case we might have chosen the wrong team for this particular game. But all the others in the season should be fine, therefore, we look for patterns and fix such outliers. Not ideal solution but helps

In [44]:
def fix_team(d):
    for index,row in d.iterrows():

        # skip the first/last 3 rows to avoid index out of range error
        if (index < 3) or (index>len(d)-3):
            continue

        # consider cases where 'team' changes from one matchweek to the next one
        if d.iloc[index,3] != d.iloc[index+1,3]:

            # the 3 ones ahead are the same as the one before (current 'team' most likely wrong)
            if (d.iloc[index+1,3] == d.iloc[index+2,3]) and (d.iloc[index+1,3] == d.iloc[index+3,3]) and (d.iloc[index+1,3] == d.iloc[index-1,3]):
                d.iloc[index,3] = d.iloc[index+1,3]

            # the 3 ones before are the same as the one after (current 'team' most likely wrong)
            elif (d.iloc[index+1,3] == d.iloc[index-1,3]) and (d.iloc[index+1,3] == d.iloc[index-2,3]) and (d.iloc[index+1,3] == d.iloc[index-3,3]):
                d.iloc[index,3] = d.iloc[index+1,3]
    return d

In [45]:
data = fix_team(data)
data = data.drop(columns=['team_check'])

In [46]:
# Few manual fixes
data.loc[(data['player'] == 'DANILO') & (data['season'] == 2019) & (data['matchweek'] == 9), 'team'] = 'JUVENTUS'
data.loc[(data['player'] == 'DANILO') & (data['season'] == 2019) & (data['matchweek'] == 18), 'team'] = 'BOLOGNA'
data.loc[(data['player'] == 'DANILO') & (data['season'] == 2019) & (data['matchweek'] == 28), 'team'] = 'BOLOGNA'
data.loc[(data['player'] == 'DANILO') & (data['season'] == 2019) & (data['matchweek'] == 37), 'team'] = 'BOLOGNA'
data.loc[(data['player'] == 'RADU') & (data['season'] == 2016)& (data['matchweek'] == 36), 'team'] = 'LAZIO'

**Merge prices table to the scores table**

In [47]:
data = pd.merge(data, price,  how='left', left_on=["player", "season", "matchweek"], right_on=["player", "season", "matchweek"])

In [48]:
print(len(data))
print(len(data[data['price'].notna()]))

98523
68349


**Make sure teams capitalization is aligned between 'team', 'home_team' and 'away_team'**

In [49]:
data['team'] = data['team'].apply(lambda x: x.upper())
data['home_team'] = data['home_team'].apply(lambda x: x.upper())
data['away_team'] = data['away_team'].apply(lambda x: x.upper())

**Convert 'match_date' to pd.datetime**

In [50]:
data['match_date'] = pd.to_datetime(data['match_date'])

**Create column opponent**
(technically feature engineering, but necessary to merge Elo Ratings below)

In [51]:
data['opponent'] = [ row['away_team'] if row['home_team']==row['team'] else row['home_team'] for index, row in data.iterrows()]

# Import Elo Ratings

### This is done separately because Elo ratings are team-based rather than player-based

In [52]:
#If you have the excel files, read them then add these columns- very important for merging on the correct season
elo_df = pd.read_excel("elo/Elo Ratings.xlsx")

In [53]:
elo_df['team'] = elo_df['team'].fillna("None").apply(lambda x : x.replace('\xa0', ''))

team_names = {"Juventus":"JUVENTUS","Inter":"INTER","SSC Napoli":"NAPOLI",
              "AS Roma":"ROMA","AC Milan":"MILAN","ACF Fiorentina":"FIORENTINA",
              "Lazio Roma":"LAZIO","Torino FC":"TORINO","Genoa CFC":"GENOA",
              "Udinese Calcio":"UDINESE","Sassuolo Calcio":"SASSUOLO",
              "Sampdoria":"SAMPDORIA","Bologna FC":"BOLOGNA","Atalanta":"ATALANTA",
              "Cagliari Calcio":"CAGLIARI","Empoli FC":"EMPOLI","Chievo Verona":"CHIEVO",
              "Pescara Calcio":"PESCARA","US Palermo":"PALERMO","FC Crotone":"CROTONE",
              "Benevento Calcio":"BENEVENTO","SPAL 2013 Ferrara":"SPAL","Hellas Verona":"VERONA",
              "Parma Calcio 1913":"PARMA","Frosinone Calcio":"FROSINONE",
              "Brescia Calcio":"BRESCIA","US Lecce":"LECCE"}

elo_df["team"] = elo_df["team"].replace(team_names)

In [54]:
# Modify date format in elo_df to match format of rest of data, compacting it into a single column and converting to datetime object

elo_df[['year', 'month', 'day']] = elo_df['Date'].str.split('/', expand=True)

elo_df['Date'] = elo_df['year'] + '-' + elo_df['month'] + '-' + elo_df['day']
elo_df['Date'] = pd.to_datetime(elo_df['Date'])

elo_df = elo_df.drop(['year','month','day'], axis=1)

In [55]:
# Rename seasons to match the format of rest of data

elo_df.loc[elo_df["Season"] == "2015-2016", "Season"] = 2015
elo_df.loc[elo_df["Season"] == "2016-2017", "Season"] = 2016
elo_df.loc[elo_df["Season"] == "2017-2018", "Season"] = 2017
elo_df.loc[elo_df["Season"] == "2018-2019", "Season"] = 2018
elo_df.loc[elo_df["Season"] == "2019-2020", "Season"] = 2019
elo_df.loc[elo_df["Season"] == "2020-2021", "Season"] = 2020
elo_df.loc[elo_df["Season"] == "2021-2022", "Season"] = 2021

elo_df["Season"] = elo_df["Season"].astype(int)

In [56]:
# Subset Elo Ratings to keep only the info we need
# Divide onto two tables as we need to merge twice: one table will provide the rating for the player's team, the other will provide the rating for its opponent in the particular matchweek

elo_df = elo_df[elo_df.Competition == "Serie A"]

elo_df.rename(columns={'Date':'match_date',
                       'Rating_before':'elo_rating',
                       'Season':'season'}, inplace=True)

team_elo = elo_df[['team', 'season', 'match_date', 'elo_rating']]

opponent_elo = elo_df[['team', 'season', 'match_date', 'elo_rating']]
opponent_elo.rename(columns={'team':'opponent',
                             'elo_rating': 'elo_rating_opponent'}, inplace=True)

/Users/jedrzejalchimowicz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [57]:
data = pd.merge(data, team_elo, how='left', left_on=["team","season","match_date"], right_on=["team", "season","match_date"])
data = pd.merge(data, opponent_elo, how='left', left_on=["opponent","season","match_date"], right_on=["opponent", "season","match_date"])

# Final Dataframe

**Sort to ensure correct calculation of rolling windows**

In [58]:
data.sort_values(by=['player', 'season', 'matchweek'], inplace=True)

In [59]:
data.head()

,player,season,matchweek,team,match_date,home_team,away_team,score,status,player_status,id,role,base_score,goal_scored,goal_conceded,penalty_saved,penalty_failed,penalty_scored,own_goal,yellow_card,red_card,assist,goal_decisive_draw,goal_decisive_win,coach,coach_score,price,opponent,elo_rating,elo_rating_opponent
0,AARONS,2017,23,VERONA,2018-02-04,VERONA,ROMA,0:1,LM,played in the game,2661.0,M,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,PECCHIA,5.0,5.0,ROMA,1771.0,2114.0
1,AARONS,2017,24,VERONA,2018-02-11,SAMPDORIA,VERONA,2:0,RW,played in the game,2661.0,M,5.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,PECCHIA,6.0,5.0,SAMPDORIA,1767.0,1935.0
2,AARONS,2017,25,VERONA,2018-02-19,LAZIO,VERONA,2:0,None,error_status,2661.0,M,5.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,PECCHIA,5.0,5.0,LAZIO,1762.0,2032.0
3,AARONS,2017,26,VERONA,2018-02-25,VERONA,TORINO,2:1,on the bench,on the bench,2661.0,M,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,4.0,TORINO,1759.0,1975.0
4,AARONS,2017,27,VERONA,2018-04-04,BENEVENTO,VERONA,3:0,on the bench,on the bench,2661.0,M,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,5.0,BENEVENTO,1765.0,1628.0


In [60]:
data.to_csv("ready_datasets/data_cleaning_output.csv")